# Imports

In [1]:
import requests
import json

# API Class

In [2]:
class OpentronsAPI():

    def __init__(self) -> None:
        self.ROBOT_IP = "169.254.241.245"
        self.HEADERS = {"opentrons-version": "3"}
        self.PIPETTE = "p300_single_gen2"
        self.runs_url = f"http://{self.ROBOT_IP}:31950/runs"
        self.lights_url = f"http://{self.ROBOT_IP}:31950/robot/lights"
        self.home_url = f"http://{self.ROBOT_IP}:31950/robot/home"
        self.protocols_url = f"http://{self.ROBOT_IP}:31950/protocols"
        self.run_id = None
        self.pipette_id = None
        self.protocol_id = None

    def post(self, url: str, headers: dict, params: str = None, data: str = None, files: list = None) -> requests.models.Response:
        r = requests.post(
            url=url,
            headers=headers,
            params=params,
            data=data,
            files=files)
        return r
    
    def get(self, url: str, headers: dict) -> requests.models.Response:
        r = requests.get(
            url=url,
            headers=headers)
        return r
    
    def display_responce(self, responce: requests.models.Response) -> None:
        json_formatted_str = json.dumps(json.loads(responce.text), indent = 2)
        print(f"Request status:\n{responce}\n{json_formatted_str}")
    
    def toggle_lights(self, verbose: bool = False) -> None:
        current_status = self.get(self.lights_url, self.HEADERS)
        current_status = json.loads(current_status.text)
        is_on = current_status['on']
        responce_data = json.dumps({"on": not(is_on)})
        toggle_responce = self.post(url = self.lights_url, headers=self.HEADERS, data=responce_data)
        if verbose:
            self.display_responce(toggle_responce)

    def create_run(self, protocol_id: str = None, verbose: bool = True) -> None:
        if protocol_id:
            protocol_id_payload = json.dumps({"data":{"protocolId": protocol_id}})
            r = self.post(self.runs_url, self.HEADERS, data = protocol_id_payload)
        else:
            r = self.post(self.runs_url, self.HEADERS)
        run_id = json.loads(r.text)['data']['id']
        self.run_id = run_id
        if verbose:
            self.display_responce(r)

    def home_robot(self, verbose: bool = True) -> None:
        command_dict = {"target": "robot"}
        command_payload = json.dumps(command_dict)
        r = self.post(url = self.home_url, headers=self.HEADERS, data = command_payload)
        if verbose:
            self.display_responce(r)

    def load_pipette(self, mount: str = 'left', verbose: bool = True) -> None:
        if self.run_id is None:
            print('No current run associated with the robot. Create a run first.')
            return
        self.commands_url = f"{self.runs_url}/{self.run_id}/commands"
        command_dict = {
            "data": {
                "commandType": "loadPipette",
                "params": {
                    "pipetteName": self.PIPETTE,
                    "mount": mount
                },
                "intent": "setup"
            }
        }
        command_payload = json.dumps(command_dict)
        r = self.post(url = self.commands_url, headers= self.HEADERS,
                      params={"waitUntilComplete": True}, data = command_payload)
        if json.loads(r.text)['data']['status'] == 'succeeded':
            self.pipette_id = json.loads(r.text)['data']['result']['pipetteId']
        
        if verbose:
            self.display_responce(r)

    def upload_protocol(self, PROTOCOL_FILE: str, LABWARE_FILE: str = None, verbose: bool = True) -> None:
        
        protocol_file_payload = open(PROTOCOL_FILE, "rb")
        files = [("files", protocol_file_payload)]
        if LABWARE_FILE:
            labware_file_payload = open(LABWARE_FILE, "rb")
            files.append(("files", labware_file_payload))

        r = self.post(url = self.protocols_url, headers = self.HEADERS, files = files)

        if verbose:
            self.display_responce(r)
        r_dict = json.loads(r.text)
        self.protocol_id = r_dict["data"]["id"]
        print(f"Protocol ID:\n{self.protocol_id}")

        protocol_file_payload.close()
        if LABWARE_FILE:
            labware_file_payload.close()

    def run_protocol(self) -> None:
        if self.run_id is None:
            print('No current run associated with the robot. Create a run with a protocol attached first.')
            return
        actions_url = f"{self.runs_url}/{self.run_id}/actions"
        action_payload = json.dumps({"data":{"actionType": "play"}})

        r = requests.post(
            url=actions_url,
            headers=self.HEADERS,
            data=action_payload
	)
        
    def move_to_coordinates(self, coordinates: tuple, min_z_height: float = 20.0, force_direct: bool = False, verbose: bool = True) -> None:
        if self.pipette_id is None:
            print('Pipette not loaded. Load pipette first.')
            return

        if len(coordinates) != 3:
            print(f'Coordinate tuple needs 3 values, got {len(coordinates)} instead.')
            return

        x,y,z = coordinates
        command_dict = {
            "data": {
                "commandType": "moveToCoordinates",
                "params": {
                    "coordinates": {"x": x, "y": y, "z": z},
                    "minimumZHeight": min_z_height,
                    "forceDirect": force_direct,
                    "pipetteId": self.pipette_id
                },
                "intent": "setup"
            }
        }

        command_payload = json.dumps(command_dict)
        r = self.post(url = self.commands_url, headers= self.HEADERS,
                      params={"waitUntilComplete": True}, data = command_payload)
        if verbose == True:
            self.display_responce(r)


    def get_all_runs(self) -> requests.models.Response:
        r = self.get(self.runs_url, self.HEADERS)
        return r
    
    def get_run_info(self) -> dict:
        all_runs_json = self.get_all_runs()
        all_runs_json = json.loads(all_runs_json.text)
        data = all_runs_json['data']
        current_run_id = None
        current_run_status = None
        for run in data:
            if run['current'] == True:
                current_run_id = run['id']
                current_run_status = run['status']
                self.run_id = current_run_id

        print(f'Total number of runs: {all_runs_json['meta']['totalLength']}')
        print(f'Current run ID: {current_run_id}')
        print(f'Current run status: {current_run_status}')
        return data

# API commands

In [3]:
openapi = OpentronsAPI()

In [4]:
openapi.toggle_lights()

In [5]:
openapi.create_run()

Request status:
<Response [201]>
{
  "data": {
    "id": "e3ae1808-09c9-4bc4-a923-68bc331fdbd0",
    "ok": true,
    "createdAt": "2024-07-09T23:11:43.521465+00:00",
    "status": "idle",
    "current": true,
    "actions": [],
    "errors": [],
    "pipettes": [],
    "modules": [],
    "labware": [],
    "liquids": [],
    "labwareOffsets": [],
    "runTimeParameters": []
  }
}


In [6]:
_ = openapi.get_run_info()

Total number of runs: 20
Current run ID: e3ae1808-09c9-4bc4-a923-68bc331fdbd0
Current run status: idle


In [14]:
runs_url = f"http://{openapi.ROBOT_IP}:31950/runs"
delete_run_url = f"{runs_url}/{openapi.run_id}"

r = requests.delete(
	url=delete_run_url,
	headers=openapi.HEADERS
	)

print(f"Request status:\n{r}\n{r.text}")

Request status:
<Response [200]>
{}


In [5]:
openapi.home_robot()

Request status:
<Response [200]>
{
  "message": "Homing robot."
}


In [6]:
openapi.load_pipette()

No current run associated with the robot. Create a run first.


In [8]:
url = f"http://{openapi.ROBOT_IP}:31950/robot/positions"
r = openapi.get(url, openapi.HEADERS)
json.loads(r.text)

{'positions': {'change_pipette': {'target': 'mount',
   'left': [300.0, 40.0, 30.0],
   'right': [95.0, 40.0, 30.0]},
  'attach_tip': {'target': 'pipette', 'point': [200.0, 90.0, 150.0]}}}

In [17]:
openapi.move_to_coordinates((200,50,170), min_z_height=1)

Request status:
<Response [201]>
{
  "data": {
    "id": "05fa1ba8-a5ef-4902-b2b9-2c4dbf3686ae",
    "createdAt": "2024-07-09T23:15:57.383055+00:00",
    "commandType": "moveToCoordinates",
    "key": "05fa1ba8-a5ef-4902-b2b9-2c4dbf3686ae",
    "status": "succeeded",
    "params": {
      "minimumZHeight": 1.0,
      "forceDirect": false,
      "pipetteId": "cebdecef-c790-4e3a-a962-8ebe365f6da2",
      "coordinates": {
        "x": 200.0,
        "y": 50.0,
        "z": 170.0
      }
    },
    "result": {
      "position": {
        "x": 200.0,
        "y": 50.0,
        "z": 170.0
      }
    },
    "startedAt": "2024-07-09T23:15:57.384884+00:00",
    "completedAt": "2024-07-09T23:15:57.717399+00:00",
    "intent": "setup",
    "notes": []
  }
}


### Load and execute protocol

In [7]:
PROTOCOL_FILE = "./protocols/test.py"
LABWARE_FILE = "./vwr_96_tiprack_200ul.json"
openapi.upload_protocol(PROTOCOL_FILE, LABWARE_FILE)

FileNotFoundError: [Errno 2] No such file or directory: './protocols/test.py'

In [28]:
openapi.create_run(protocol_id=openapi.protocol_id)

Request status:
<Response [201]>
{
  "data": {
    "id": "f6a256ac-261c-41f6-ba4e-0692bf2055c4",
    "ok": true,
    "createdAt": "2024-07-09T23:42:14.581162+00:00",
    "status": "idle",
    "current": true,
    "actions": [],
    "errors": [],
    "pipettes": [],
    "modules": [],
    "labware": [],
    "liquids": [],
    "labwareOffsets": [],
    "runTimeParameters": [],
    "protocolId": "5027fe50-5f0a-423c-9ce4-2b2d8e18e3df"
  }
}


In [29]:
openapi.run_protocol()

In [22]:
protocols_url = f"http://{openapi.ROBOT_IP}:31950/protocols"

r = requests.get(
	url=protocols_url,
	headers=openapi.HEADERS
	)

protocols_dict = json.loads(r.text)
print("Protocols list:")
for p in protocols_dict["data"]:
	print(p)

Protocols list:
{'id': 'dafb7d77-8245-448d-9864-27fa1e3137d5', 'createdAt': '2024-05-14T22:57:28.389017+00:00', 'files': [{'name': 'test_vwr_96_tiprack_200ul.py', 'role': 'main'}, {'name': 'vwr_96_tiprack_200ul.json', 'role': 'labware'}], 'protocolType': 'python', 'robotType': 'OT-2 Standard', 'metadata': {'apiLevel': '2.0'}, 'analyses': [], 'analysisSummaries': [{'id': 'a9ee0d6c-94d5-4001-894e-e31f6a51450c', 'status': 'completed'}], 'key': 'a90763dc-bb0f-4f8b-849d-3e5b31cf6a46'}
{'id': 'e40e72fa-9362-4434-adcc-b72c6c67ed71', 'createdAt': '2024-05-14T23:13:32.939907+00:00', 'files': [{'name': 'test_vwr_96_tiprack_200ul.py', 'role': 'main'}, {'name': 'vwr_96_tiprack_200ul.json', 'role': 'labware'}], 'protocolType': 'python', 'robotType': 'OT-2 Standard', 'metadata': {'apiLevel': '2.0'}, 'analyses': [], 'analysisSummaries': [{'id': '24db9ff8-f373-48cd-807b-9a07a89882e4', 'status': 'completed'}], 'key': '918a57f2-1e17-4735-af45-cd1bcad0aef9'}
{'id': '3047424e-0c54-45e0-8783-1444f0e75e26',

### Liquid pickup attempt

In [9]:
TIP_RACK = "opentrons_96_tiprack_300ul"
TIP_RACK2 = "vwr_96_tiprack_200ul"
RESERVOIR = "nest_12_reservoir_15ml"
WELL_PLATE = "corning_96_wellplate_360ul_flat"

In [12]:

LABWARE_FILE = "./vwr_96_tiprack_200ul.json"

HEADERS = {"opentrons-version": "3"}

protocols_url = f"http://{openapi.ROBOT_IP}:31950/protocols"

labware_file_payload = open(LABWARE_FILE, "rb")

r = requests.post(
	url=protocols_url,
	headers=HEADERS,
	files=[("files", labware_file_payload)]
	)

r_dict = json.loads(r.text)

labware_file_payload.close()

In [13]:
r_dict

{'errors': [{'id': 'ProtocolFilesInvalid',
   'title': 'Protocol File(s) Invalid',
   'detail': 'Error 4006 INVALID_PROTOCOL_DATA (RoleAnalysisError): "vwr_96_tiprack_200ul.json" is not a valid protocol file.',
   'errorCode': '4000'}]}

In [10]:
# Load labware 1 (tip rack)
command_dict = {
	"data": {
		"commandType": "loadLabware",
		"params": {
			"location": {"slotName": "1"},
			"loadName": TIP_RACK2,
			"namespace": "opentrons",
			"version": 1
		},
		"intent": "setup"
	}
}

command_payload = json.dumps(command_dict)
print(f"Command:\n{command_payload}")

r = requests.post(
	url=openapi.commands_url,
	headers=openapi.HEADERS,
	params={"waitUntilComplete": True},
	data=command_payload
	)

r_dict = json.loads(r.text)
labware_1_id = r_dict["data"]["result"]["labwareId"]
print(f"Labware 1 (tip rack) ID:\n{labware_1_id}\n")

Command:
{"data": {"commandType": "loadLabware", "params": {"location": {"slotName": "1"}, "loadName": "vwr_96_tiprack_200ul", "namespace": "opentrons", "version": 1}, "intent": "setup"}}


KeyError: 'result'

In [11]:
r_dict

{'data': {'id': '45a944ee-6211-4a26-b2ea-18bc6d86879d',
  'createdAt': '2024-07-09T20:57:16.238835+00:00',
  'commandType': 'loadLabware',
  'key': '45a944ee-6211-4a26-b2ea-18bc6d86879d',
  'status': 'failed',
  'params': {'location': {'slotName': '1'},
   'loadName': 'vwr_96_tiprack_200ul',
   'namespace': 'opentrons',
   'version': 1},
  'error': {'id': '4ba83869-85b5-4e04-81e8-76df95c066d8',
   'createdAt': '2024-07-09T20:57:16.281830+00:00',
   'errorCode': '4000',
   'errorType': 'PythonException',
   'detail': 'FileNotFoundError: Labware "vwr_96_tiprack_200ul" not found with version 1 in namespace "opentrons".',
   'errorInfo': {'args': '(\'Labware "vwr_96_tiprack_200ul" not found with version 1 in namespace "opentrons".\',)',
    'errno': 'None',
    'filename': 'None',
    'filename2': 'None',
    'strerror': 'None',
    'traceback': '  File "/usr/lib/python3.10/site-packages/opentrons/protocol_engine/execution/command_executor.py", line 150, in execute\n\n  File "/usr/lib/pyth

In [21]:
# Load labware 2 (reservoir)
command_dict = {
	"data": {
		"commandType": "loadLabware",
		"params": {
			"location": {"slotName": "3"},
			"loadName": WELL_PLATE,
			"namespace": "opentrons",
			"version": 1
		},
		"intent": "setup"
	}
}

command_payload = json.dumps(command_dict)
print(f"Command:\n{command_payload}")

r = requests.post(
	url=openapi.commands_url,
	headers=openapi.HEADERS,
	params={"waitUntilComplete": True},
	data=command_payload
	)

r_dict = json.loads(r.text)
labware_2_id = r_dict["data"]["result"]["labwareId"]
print(f"Labware 2 (reservoir) ID:\n{labware_2_id}\n")

Command:
{"data": {"commandType": "loadLabware", "params": {"location": {"slotName": "3"}, "loadName": "corning_96_wellplate_360ul_flat", "namespace": "opentrons", "version": 1}, "intent": "setup"}}
Labware 2 (reservoir) ID:
b7bc5044-bf0e-40f0-afc5-b4d10a037596



In [22]:
command_dict = {
	"data": {
		"commandType": "loadLabware",
		"params": {
			"location": {"slotName": "2"},
			"loadName": "nest_1_reservoir_290ml",
			"namespace": "opentrons",
			"version": 1
		},
		"intent": "setup"
	}
}

command_payload = json.dumps(command_dict)
print(f"Command:\n{command_payload}")

r = requests.post(
	url=openapi.commands_url,
	headers=openapi.HEADERS,
	params={"waitUntilComplete": True},
	data=command_payload
	)

r_dict = json.loads(r.text)
labware_3_id = r_dict["data"]["result"]["labwareId"]
print(f"Labware 3 (reservoir) ID:\n{labware_3_id}\n")

Command:
{"data": {"commandType": "loadLabware", "params": {"location": {"slotName": "2"}, "loadName": "nest_1_reservoir_290ml", "namespace": "opentrons", "version": 1}, "intent": "setup"}}
Labware 3 (reservoir) ID:
8c039e2a-5bd8-40d1-bd3b-3c06ea571ac9



In [24]:
command_dict = {
	"data": {
		"commandType": "pickUpTip",
		"params": {
			"labwareId": labware_1_id,
			"wellName": "A1",
			"wellLocation": {
				"origin": "top", "offset": {"x": 0, "y": 0, "z": 0}
			},
			"pipetteId": openapi.pipette_id
		},
		"intent": "setup"
	}
}

command_payload = json.dumps(command_dict)
print(f"Command:\n{command_payload}\n")

r = requests.post(
	url=openapi.commands_url,
	headers=openapi.HEADERS,
	data=command_payload
	)

print(f"Response:\n{r}\n{r.text}\n")

Command:
{"data": {"commandType": "pickUpTip", "params": {"labwareId": "35649278-ee0c-412e-8a6b-c1e6f6144d68", "wellName": "A1", "wellLocation": {"origin": "top", "offset": {"x": 0, "y": 0, "z": 0}}, "pipetteId": "e2534277-8a57-4760-8ea8-aebbdf19aa17"}, "intent": "setup"}}

Response:
<Response [201]>
{"data": {"id": "a7fc69bd-20bc-4fcf-bb9c-0b2ad299b229", "createdAt": "2024-06-14T07:11:31.853808+00:00", "commandType": "pickUpTip", "key": "a7fc69bd-20bc-4fcf-bb9c-0b2ad299b229", "status": "queued", "params": {"labwareId": "35649278-ee0c-412e-8a6b-c1e6f6144d68", "wellName": "A1", "wellLocation": {"origin": "top", "offset": {"x": 0.0, "y": 0.0, "z": 0.0}}, "pipetteId": "e2534277-8a57-4760-8ea8-aebbdf19aa17"}, "intent": "setup"}}



In [29]:
# openapi.move_to_coordinates((200,50,50), min_z_height=1)
# openapi.move_to_coordinates((200,50,5), min_z_height=1)


command_dict = {
	"data": {
		"commandType": "aspirate",
		"params": {
			"labwareId": labware_3_id,
			"wellName": "A1",
			"wellLocation": {
				"origin": "top", "offset": {"x": 0, "y": 0, "z": -40}
			},
			"flowRate": 10,
			"volume": 100,
			"pipetteId": openapi.pipette_id
		},
		"intent": "setup"
	}
}

command_payload = json.dumps(command_dict)
print(f"Command:\n{command_payload}\n")

r = requests.post(
	url=openapi.commands_url,
	headers=openapi.HEADERS,
	data=command_payload,
    params={"waitUntilComplete": True}
	)

print(f"Response:\n{r}\n{r.text}\n")

Command:
{"data": {"commandType": "aspirate", "params": {"labwareId": "8c039e2a-5bd8-40d1-bd3b-3c06ea571ac9", "wellName": "A1", "wellLocation": {"origin": "top", "offset": {"x": 0, "y": 0, "z": -40}}, "flowRate": 10, "volume": 100, "pipetteId": "e2534277-8a57-4760-8ea8-aebbdf19aa17"}, "intent": "setup"}}

Response:
<Response [201]>
{"data": {"id": "b24f6040-137d-4a03-9e4f-84e973fb1c13", "createdAt": "2024-06-14T07:13:52.252198+00:00", "commandType": "aspirate", "key": "b24f6040-137d-4a03-9e4f-84e973fb1c13", "status": "succeeded", "params": {"labwareId": "8c039e2a-5bd8-40d1-bd3b-3c06ea571ac9", "wellName": "A1", "wellLocation": {"origin": "top", "offset": {"x": 0.0, "y": 0.0, "z": -40.0}}, "flowRate": 10.0, "volume": 100.0, "pipetteId": "e2534277-8a57-4760-8ea8-aebbdf19aa17"}, "result": {"position": {"x": 196.38, "y": 42.74, "z": 4.399999999999997}, "volume": 100.0}, "startedAt": "2024-06-14T07:13:52.253813+00:00", "completedAt": "2024-06-14T07:14:06.759121+00:00", "intent": "setup"}}



In [30]:
command_dict = {
	"data": {
		"commandType": "dispense",
		"params": {
			"labwareId": labware_2_id,
			"wellName": "A2",
			"wellLocation": {
				"origin": "top", "offset": {"x": 0, "y": 0, "z": 0}
			},
			"flowRate": 10,
			"volume": 100,
			"pipetteId": openapi.pipette_id
		},
		"intent": "setup"
	}
}

command_payload = json.dumps(command_dict)
print(f"Command:\n{command_payload}\n")

r = requests.post(
	url=openapi.commands_url,
	headers=openapi.HEADERS,
	data=command_payload,
    params={"waitUntilComplete": True}
	)

print(f"Response:\n{r}\n{r.text}\n")

Command:
{"data": {"commandType": "dispense", "params": {"labwareId": "b7bc5044-bf0e-40f0-afc5-b4d10a037596", "wellName": "A2", "wellLocation": {"origin": "top", "offset": {"x": 0, "y": 0, "z": 0}}, "flowRate": 10, "volume": 100, "pipetteId": "e2534277-8a57-4760-8ea8-aebbdf19aa17"}, "intent": "setup"}}

Response:
<Response [201]>
{"data": {"id": "4386471b-66b2-4395-a355-a151021e5adc", "createdAt": "2024-06-14T07:14:06.843683+00:00", "commandType": "dispense", "key": "4386471b-66b2-4395-a355-a151021e5adc", "status": "succeeded", "params": {"labwareId": "b7bc5044-bf0e-40f0-afc5-b4d10a037596", "wellName": "A2", "wellLocation": {"origin": "top", "offset": {"x": 0.0, "y": 0.0, "z": 0.0}}, "flowRate": 10.0, "volume": 100.0, "pipetteId": "e2534277-8a57-4760-8ea8-aebbdf19aa17"}, "result": {"position": {"x": 288.38, "y": 74.24, "z": 14.219999999999999}, "volume": 100.0}, "startedAt": "2024-06-14T07:14:06.845252+00:00", "completedAt": "2024-06-14T07:14:19.409160+00:00", "intent": "setup"}}



In [31]:
command_dict = {
	"data": {
		"commandType": "blowout",
		"params": {
			"labwareId": labware_2_id,
			"wellName": "A2",
			"wellLocation": {
				"origin": "top", "offset": {"x": 0, "y": 0, "z": 0}
			},
			"flowRate": 10,
			"pipetteId": openapi.pipette_id
		},
		"intent": "setup"
	}
}

command_payload = json.dumps(command_dict)
print(f"Command:\n{command_payload}\n")

r = requests.post(
	url=openapi.commands_url,
	headers=openapi.HEADERS,
	data=command_payload,
    params={"waitUntilComplete": True}
	)

print(f"Response:\n{r}\n{r.text}\n")

Command:
{"data": {"commandType": "blowout", "params": {"labwareId": "b7bc5044-bf0e-40f0-afc5-b4d10a037596", "wellName": "A2", "wellLocation": {"origin": "top", "offset": {"x": 0, "y": 0, "z": 0}}, "flowRate": 10, "pipetteId": "e2534277-8a57-4760-8ea8-aebbdf19aa17"}, "intent": "setup"}}

Response:
<Response [201]>
{"data": {"id": "25f080f9-bf07-4ea1-831f-f2e6c9e661f3", "createdAt": "2024-06-14T07:14:19.467575+00:00", "commandType": "blowout", "key": "25f080f9-bf07-4ea1-831f-f2e6c9e661f3", "status": "succeeded", "params": {"labwareId": "b7bc5044-bf0e-40f0-afc5-b4d10a037596", "wellName": "A2", "wellLocation": {"origin": "top", "offset": {"x": 0.0, "y": 0.0, "z": 0.0}}, "flowRate": 10.0, "pipetteId": "e2534277-8a57-4760-8ea8-aebbdf19aa17"}, "result": {"position": {"x": 288.38, "y": 74.24, "z": 14.219999999999999}}, "startedAt": "2024-06-14T07:14:19.469225+00:00", "completedAt": "2024-06-14T07:14:23.883102+00:00", "intent": "setup"}}



In [32]:
command_dict = {
	"data": {
		"commandType": "dropTip",
		"params": {
			"labwareId": labware_1_id,
			"wellName": "A1",
			"wellLocation": {
				"origin": "top", "offset": {"x": 0, "y": 0, "z": 0}
			},
			"pipetteId": openapi.pipette_id
		},
		"intent": "setup"
	}
}

command_payload = json.dumps(command_dict)
print(f"Command:\n{command_payload}\n")

r = requests.post(
	url=openapi.commands_url,
	headers=openapi.HEADERS,
	data=command_payload,
    params={"waitUntilComplete": True}
	)

print(f"Response:\n{r}\n{r.text}\n")

Command:
{"data": {"commandType": "dropTip", "params": {"labwareId": "35649278-ee0c-412e-8a6b-c1e6f6144d68", "wellName": "A1", "wellLocation": {"origin": "top", "offset": {"x": 0, "y": 0, "z": 0}}, "pipetteId": "e2534277-8a57-4760-8ea8-aebbdf19aa17"}, "intent": "setup"}}

Response:
<Response [201]>
{"data": {"id": "3e0af3fb-2557-470a-87c7-f63c4822a335", "createdAt": "2024-06-14T07:30:30.261117+00:00", "commandType": "dropTip", "key": "3e0af3fb-2557-470a-87c7-f63c4822a335", "status": "succeeded", "params": {"pipetteId": "e2534277-8a57-4760-8ea8-aebbdf19aa17", "labwareId": "35649278-ee0c-412e-8a6b-c1e6f6144d68", "wellName": "A1", "wellLocation": {"origin": "top", "offset": {"x": 0.0, "y": 0.0, "z": 0.0}}, "alternateDropLocation": false}, "result": {"position": {"x": 14.38, "y": 74.24, "z": 64.69}}, "startedAt": "2024-06-14T07:30:30.262871+00:00", "completedAt": "2024-06-14T07:30:43.309438+00:00", "intent": "setup"}}



# Drafts

### New run

In [4]:
ROBOT_IP = "169.254.241.245"
HEADERS = {"opentrons-version": "3"}
PIPETTE = "p300_single_gen2"

runs_url = f"http://{ROBOT_IP}:31950/runs"
print(f"Command:\n{runs_url}")

r = requests.post(
	url=runs_url,
	headers=HEADERS
	)

r_dict = json.loads(r.text)
run_id = r_dict["data"]["id"]
print(f"Run ID:\n{run_id}")

Command:
http://169.254.241.245:31950/runs
Run ID:
4c6983d8-a7d1-48d8-8d3d-2d673a28de45


In [43]:
dct = json.loads(r.text)

In [47]:
dct

{'data': [{'id': '004afff6-abb6-41a9-83ea-af51b27c6679',
   'createdAt': '2024-05-14T22:57:29.228454+00:00',
   'status': 'stopped',
   'current': False,
   'actions': [{'id': 'f90cf8ec-2cfd-4a53-b424-42e876289d00',
     'createdAt': '2024-05-14T22:59:01.712987+00:00',
     'actionType': 'stop'}],
   'errors': [],
   'pipettes': [],
   'modules': [],
   'labware': [{'id': 'fixedTrash',
     'loadName': 'opentrons_1_trash_1100ml_fixed',
     'definitionUri': 'opentrons/opentrons_1_trash_1100ml_fixed/1',
     'location': {'slotName': '12'}}],
   'liquids': [],
   'labwareOffsets': [],
   'protocolId': 'dafb7d77-8245-448d-9864-27fa1e3137d5',
   'completedAt': '2024-05-14T22:59:07.519077+00:00'},
  {'id': 'ab5ad4a9-1402-4363-8e24-2d7b92848419',
   'createdAt': '2024-05-14T23:13:33.743584+00:00',
   'status': 'stopped',
   'current': False,
   'actions': [{'id': 'be6c7173-a7f6-42b1-b42f-3362f86a802a',
     'createdAt': '2024-05-14T23:14:40.905255+00:00',
     'actionType': 'play'},
    {'id

In [48]:
for thing in dct['data']:
    if thing['current'] == True:
        print(thing['id'])

1137fe37-7fe4-4115-b904-4a6d50cdf7a6


### Get position

In [53]:
positions_url = f"http://{openapi.ROBOT_IP}:31950/robot/positions"
r = requests.get(
    url=positions_url,
    headers=openapi.HEADERS)
json.loads(r.text)

{'positions': {'change_pipette': {'target': 'mount',
   'left': [300.0, 40.0, 30.0],
   'right': [95.0, 40.0, 30.0]},
  'attach_tip': {'target': 'pipette', 'point': [200.0, 90.0, 150.0]}}}

### Setup pipette

In [6]:
commands_url = f"{runs_url}/{run_id}/commands"

command_dict = {
	"data": {
		"commandType": "loadPipette",
		"params": {
			"pipetteName": PIPETTE,
			"mount": "left"
		},
		"intent": "setup"
	}
}

command_payload = json.dumps(command_dict)
print(f"Command:\n{command_payload}")

r = requests.post(
	url=commands_url,
	headers=HEADERS,
	params={"waitUntilComplete": True},
	data=command_payload
	)

r_dict = json.loads(r.text)
pipette_id = r_dict["data"]["result"]["pipetteId"]
print(f"Pipette ID:\n{pipette_id}\n")

Command:
{"data": {"commandType": "loadPipette", "params": {"pipetteName": "p300_single_gen2", "mount": "left"}, "intent": "setup"}}
Pipette ID:
1b1c1005-4367-4eef-9d01-f5f968c63006



### Home robot

In [7]:
home_url = f"http://{ROBOT_IP}:31950/robot/home"
command_dict = {"target": "robot"}
command_payload = json.dumps(command_dict)
print(f"Command:\n{command_payload}")

r = requests.post(
	url=home_url,
	headers=HEADERS,
	data=command_payload
	)

print(f"Response:\n{r}\n{r.text}\n")

Command:
{"target": "robot"}
Response:
<Response [200]>
{"message":"Homing robot."}



In [18]:
move_url = f"http://{ROBOT_IP}:31950/robot/move"
command_dict = {"target":"pipette",
                "point":[100,100,80],
                "mount":"left"}
command_payload = json.dumps(command_dict)

r = requests.post(
	url=home_url,
	headers=HEADERS,
	data=command_payload
	)

print(f"Response:\n{r}\n{r.text}\n")

Response:
<Response [200]>
{"message":"Pipette on left homed successfully"}



In [9]:
command_dict = {
	"data": {
		"commandType": "moveToCoordinates",
		"params": {
			"coordinates": {"x": 200, "y": 100, "z": 150},
			"minimumZHeight": 20,
			"forceDirect": True,
			"pipetteId": pipette_id
		},
		"intent": "setup"
	}
}

command_payload = json.dumps(command_dict)
print(f"Command:\n{command_payload}\n")

r = requests.post(
	url=commands_url,
	headers=HEADERS,
	data=command_payload,
    params={"waitUntilComplete": True}
	)

json.loads(r.text)
# print(f"Response:\n{r}\n{r.text}\n")

Command:
{"data": {"commandType": "moveToCoordinates", "params": {"coordinates": {"x": 200, "y": 100, "z": 150}, "minimumZHeight": 20, "forceDirect": true, "pipetteId": "1b1c1005-4367-4eef-9d01-f5f968c63006"}, "intent": "setup"}}



{'data': {'id': '3f13ec65-ddcd-4b84-ab0c-19a4f42c8ff7',
  'createdAt': '2024-06-14T03:26:26.246412+00:00',
  'commandType': 'moveToCoordinates',
  'key': '3f13ec65-ddcd-4b84-ab0c-19a4f42c8ff7',
  'status': 'succeeded',
  'params': {'minimumZHeight': 20.0,
   'forceDirect': True,
   'pipetteId': '1b1c1005-4367-4eef-9d01-f5f968c63006',
   'coordinates': {'x': 200.0, 'y': 100.0, 'z': 150.0}},
  'result': {'position': {'x': 200.0, 'y': 100.0, 'z': 150.0}},
  'startedAt': '2024-06-14T03:26:26.247949+00:00',
  'completedAt': '2024-06-14T03:26:26.780252+00:00',
  'intent': 'setup'}}